In [2]:
import string
import joblib
from os import path

import numpy as np
import pandas as pd
from scipy import sparse
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm

from utils import ls, cat
from pathlib import Path

from time import time

import requests

tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

tqdm.pandas()

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
#Dmitry
# DATA_DIR = '../../data'
# OUTPUT_DIR = '../../data'

#Anton
DATA_DIR = Path('D:/data/lenta')
OUTPUT_DIR = Path('D:/data/lenta')

#Dmitry
# TRAIN_PATH = f'{DATA_DIR}/lenta-ru-news.csv'

#Anton
TRAIN_PATH = Path('D:/data/lenta/lenta-ru-news.csv/lenta-ru-news.csv')
NUM_ROWS = 10
TRAIN_LEMMAS_PATH = OUTPUT_DIR / 'train_lemmas.csv'
TRAIN_LEMMAS_PATH_ALL = OUTPUT_DIR / 'train_lemmas_all.csv'


ls(DATA_DIR)

DIR      D:\data\lenta\lenta-ru-news.csv
611.9 MB D:\data\lenta\lenta-ru-news.csv.zip
  7.8 MB D:\data\lenta\train_lemmas.csv


In [4]:
df_train = pd.read_csv(TRAIN_PATH, nrows=NUM_ROWS)
# df_train = pd.read_csv(TRAIN_PATH)

display(df_train.head(3))
display(df_train.shape)

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17


(10, 6)

## Features

In [5]:
text_cols = ['title', 'text']
all_colls = ['title', 'text', 'date']
text_cols, all_colls

(['title', 'text'], ['title', 'text', 'date'])

In [6]:
df_train[all_colls].head()


,title,text,date
0,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,1914/09/16
1,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",1914/09/16
2,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",1914/09/17
3,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,1914/09/17
4,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",1914/09/18


## Tokenized Text

In [7]:
# !pip install spacy
# python -m spacy download ru_core_news_md

In [8]:
import spacy

nlp = spacy.load('ru_core_news_md', disable=['tagger', 'attribute_ruler', 'senter', 'parser', 'ner'])

In [9]:
df_train[text_cols]

,title,text
0,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...
1,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про..."
2,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в..."
3,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...
4,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю..."
5,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...
6,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...
7,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...
8,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...
9,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...


In [10]:
# train_tokens = df_train[text_cols].fillna('') \
#     .progress_applymap(nlp)

train_tokens = pd.DataFrame()

In [12]:
for i in text_cols:
    print(i)
    # df = pd.read_csv(TRAIN_PATH,  usecols=[i], nrows=NUM_ROWS)
    df = pd.read_csv(TRAIN_PATH,  usecols=[i])
    train_tokens[i] = df[i].fillna('') \
        .parallel_apply(nlp)
    del df


title


In [ ]:
train_tokens


In [ ]:
def is_alphanum(token):
    return not token.is_punct \
        and not token.is_currency \
        and not token.is_digit \
        and not token.is_punct \
        and not token.is_oov \
        and not token.is_space \
        and not token.is_stop \
        and not token.like_num \
        and not token.pos_ == "PROPN"

df_text = train_tokens \
    .progress_applymap(lambda x: ' '.join(t.lemma_ for t in x if is_alphanum(t)))

df_output_train = pd.concat([df_text, df_train['date']], axis=1)

df_output_train.to_csv(TRAIN_LEMMAS_PATH)

In [ ]:
# ls(OUTPUT_DIR, path.basename(TRAIN_LEMMAS_PATH))
ls(OUTPUT_DIR, path.basename(TRAIN_LEMMAS_PATH_ALL))

In [ ]:
df = pd.read_csv(TRAIN_LEMMAS_PATH, index_col=[0])
df['date'] = df['date'].astype('datetime64')
df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

n_features = 1000

data_samples = df.title.fillna('')
data_samples

In [ ]:

# Use tf-idf features for NMF.
print("Extracting tf-idf features")
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2, max_features=n_features)

t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for")
tf_vectorizer = CountVectorizer(
    max_df=0.95, min_df=2, max_features=n_features)

t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

In [ ]:
n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20
batch_size = 128
init = "nndsvda"

def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, learning_method="batch",
                                max_iter=25, random_state=0, n_jobs=-1)


print(
    "\n" * 2,
    "Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
    % (n_samples, n_features),
)

t0 = time()
doc_topics = lda.fit_transform(tf)
print("done in %0.3fs." % (time() - t0))

tf_feature_names = tf_vectorizer.get_feature_names_out()
plot_top_words(lda, tf_feature_names, n_top_words, "Topics in LDA model")

In [ ]:
df.date.hist()

In [ ]:
df_train = pd.read_csv(TRAIN_PATH)

In [ ]:
df_train

In [ ]:
df_train['date'] = df["date"].astype("datetime64")

In [ ]:
print('Рапределение дат в исходном dataset')
df_train.my.groupby([df["date"].dt.year, df["date"].dt.month]).count().plot(kind="bar")

In [ ]:
print('Количество строк с датой в 1914 году:')
print(df_train[df_train.date < '1950-01-01'].date.count())

In [ ]:
df.drop(df[df.date < '1950-01-01'].index, inplace=True)

In [ ]:
from pydoc_data.topics import topics


df_topic0 = df[df.topic ==0].groupby(by='date').count()

In [ ]:
df_topic0

In [ ]:
df_topic0.plot(y='topic',title='topic 0')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))

for i in range(0, 10):
    x = df[df.topic == i].groupby(by='date').count().index.to_numpy()
    y = df[df.topic == i].groupby(by='date').count().topic.to_numpy()
    plt.plot(x, y, label = 'topic' + str(i))


plt.legend()
plt.title('All topics')
plt.show()

In [ ]:
topic_num = 2
date_list = df[df.topic == topic_num].groupby(by='date').count().index.to_numpy()
count_list = df[df.topic == topic_num].groupby(by='date').count().topic.to_numpy()

fig, ax = plt.subplots()
fig.set_size_inches(20, 10)
# ax = plt.gca()

def plot_changes_(ax, date_list, count_list, thresh=.4):

    thresh = 0.4
    topic_num = 0
    # date_list[:10], count_list[:120]

    norm_count = [float(i) / max(count_list) for i in count_list]
    ax.plot(date_list, norm_count, label='norm')

    changes = []
    for i in range(1, len(norm_count)):
        changes.append(norm_count[i] - norm_count[i-1])
        
    ax.plot(date_list[1:], changes, label='changes')
    ind_great_changes = [i+1 for i,v in enumerate(changes) if v >thresh]
    ax.scatter(np.array(date_list)[ind_great_changes], np.array(norm_count)[ind_great_changes], c='r', label='date with great changes')

    # ax.title('Даты с изменением в процентных изменения максимума более чем на {}%'.format(thresh *100))
    ax.legend()
    
    
plot_changes_(ax, date_list, count_list)

In [ ]:
fig, axes = plt.subplots(5, 2)

fig.set_size_inches(20, 20)

ax = axes.flatten()

for i in range(0, 10):
    topic_num = i
    date_list = df[df.topic == topic_num].groupby(by='date').count().index.to_numpy()
    count_list = df[df.topic == topic_num].groupby(by='date').count().topic.to_numpy()
    plot_changes_(ax[i], date_list, count_list)

In [ ]:
topic_num = 0
thresh = 0.4

date_list = df[df.topic == topic_num].groupby(by='date').count().index.to_numpy()
count_list = df[df.topic == topic_num].groupby(by='date').count().topic.to_numpy()
norm_count = [float(i) / max(count_list) for i in count_list]
changes = []

for i in range(1, len(norm_count)):
    changes.append(norm_count[i] - norm_count[i-1])

ind_great_changes = [i+1 for i,v in enumerate(changes) if v >thresh]

date_great_changes = np.array(date_list)[ind_great_changes]
for date in date_great_changes:
    print(date)
    ind = df[(df.date == date) & (df.topic ==  topic_num)].index
    print(df_train.loc[ind].title)

In [ ]:
date0 = date_great_changes[0]
date0

In [ ]:
df_train[]

In [ ]:
print(df_train.loc[ind].title)